# softmax分类器

softmax regression是logisitic regression在多分类问题上的推广。softmax可以认为是一组超平面的集合，每个超平面代表一个类，每个类的参数可以由*法向量*$\mathbf{\theta}$和*截距*$b$来描述。在分类时，我们计算每个类别的得分$\exp(\mathbf{\theta}_k^Tx+b_k),k=1,2,...,C$，然后把样本分配到得分最高的类。通常我们将这个得分进行归一化转化为一个概率分布：
$$P(Y=k|x,\mathbf{\Theta},\mathbf{b})= \frac{\exp(\mathbf{\theta}_k^Tx+b_k)}{\sum_{k=1}^C \exp(\mathbf{\theta}_k^Tx+b_k)}$$

Notations:
* $x$:输入向量，$d\times 1$列向量，$d$是feature数
* $W=[w_1,w_2,...,w_c]^T$:权重矩阵，$c\times d$矩阵，$c$是label数，每一行对应一个类的超平面法向量
* $b$：每个类对应超平面的偏置组成的向量， $c\times 1$列向量
* $z=Wx+b$:线性分类器输出， $c\times 1$列向量
* $\hat{y}$:softmax函数输出， $c\times 1$列向量
* 记$\vec{e}_j=[0,...,1,...,0]^T\in\mathbb{R}^{c\times 1}$，其中$1$出现在第$j$个位置
* $1_c$表示一个全$1$的$c$维列向量
* $y$：我们要拟合的目标变量，是一个one-hot vector（只有一个1，其余均为0），也是 $c\times 1$列向量 。 我们将其转置，表示为一个列向量：$y=(0,...,1,...,0)^T$

他们之间的关系：
$$\left\{\begin{aligned}&z=Wx+b\\& \hat{y}=\mathrm{softmax}(z)=\frac{exp(z)}{1_c^Texp(z)} \end{aligned}\right.$$

cross-entropy error定义为：
$$ E = -y^Tlog(\hat{y}) $$
因为$y$是一个one-hot vector（即只有一个位置为1），假设$y_k=1$，那么上式等于$-log(\hat{y}_k)=-log(\frac{exp(z_k)}{\sum\limits_i exp(z_i)})=-z_k+log(\sum\limits_i exp(z_i))$


依据chain rule有：
$$ \begin{aligned}\frac{\partial E}{\partial W_{ij}}
&=tr\bigg(\big(\frac{\partial E}{\partial z}\big)^T\frac {\partial z}{\partial W_{ij}}\bigg)\\
&=tr\bigg( \big(\color{red}{\frac{\partial \hat{y}}{\partial z}\cdot\frac{\partial E}{\partial \hat{y}}}\big)^T\frac {\partial z}{\partial W_{ij}} \bigg)\end{aligned}$$
注意上式中红色部分的相乘顺序，这里我用了`Denominator layout`，因此套用链式法则时要颠倒相乘顺序，原先是
$$E\to \hat{y}, \hat{y} \to z$$
用了`Denominator layout`后要反过来，变为了
$$\hat{y} \to z , E\to \hat{y}$$



我们一个一个来求上式中的各个部分
1. $\frac{\partial \hat{y}}{\partial z}$
$$\begin{equation}\begin{aligned}\frac{\partial \hat{y}}{\partial z}&=\frac{\partial ( \frac{exp(z)}{1_c^Texp(z)})}{\partial z}\\&= \frac{1}{1_c^Texp(z)}\frac{\partial exp(z)}{\partial z}+ \frac{\partial (\frac{1}{1_c^Texp(z)})}{\partial z}( exp(z) )^T\\&= \frac{1}{1_c^Texp(z)}diag(exp(z))-\frac{1}{(1_c^Texp(z))^2}exp(z)exp(z)^T\\&=diag(\frac{exp(z)}{1_c^Texp(z)})-\frac{exp(z)}{1_c^Texp(z)}\cdot (\frac{exp(z)}{1_c^Texp(z)})^T\\&=diag(\mathrm{ softmax}(z))- \mathrm{ softmax}(z) \mathrm{ softmax}(z)^T\\&=diag(\hat{y})-\hat{y}\hat{y}^T \end{aligned}\end{equation}$$
注：上述求导过程使用了`Denominator layout`。
设$a=a( \boldsymbol{ x}),\boldsymbol{u}= \boldsymbol{u}( \boldsymbol{x}) $，这里$ \boldsymbol{ x}$特意加粗表示是列向量，$a$没加粗表示是一个标量函数，$ \boldsymbol{u}$加粗表示是一个向量函数。在`Numerator layout`下，$\frac{\partial a \boldsymbol{u}}{ \boldsymbol{x}}=a\frac{\partial  \boldsymbol{u}}{\partial  \boldsymbol{x}}+ \boldsymbol{u}\frac{\partial a}{\partial  \boldsymbol{x}} $，而在`Denominator layout`下，则为$\frac{\partial a \boldsymbol{u}}{\partial  \boldsymbol{x}}=a\frac{\partial  \boldsymbol{u}}{\partial  \boldsymbol{x}}+\frac{\partial a}{\partial  \boldsymbol{x}} \boldsymbol{u}^T$，对比可知上述推导用的实际是`Denominator layout`。
以下推导均采用 Denominator layout，这样的好处是我们用梯度更新权重时不需要对梯度再转置。

2. $\frac{\partial E}{\partial \hat{y}}$  
同样应用反向链式法则有：
$$\begin{equation}\frac{\partial E}{\partial \hat{y}}=\frac{\partial log(\hat{y})}{\partial \hat{y}}\cdot \frac{\partial (-y^Tlog(\hat{y}))}{\partial log(\hat{y})}=\big(diag(\hat{y})\big)^{-1}\cdot(-y)\end{equation}$$

3. $\frac{\partial z}{\partial W_{ij}} $  
$z$的第$k$个分量可以表示为：$z_k=\sum\limits_j W_{kj}x_j+b_k$，因此
$$\begin{equation}\frac{\partial z}{\partial W_{ij}} =\begin{bmatrix}\frac{\partial z_1}{\partial W_{ij}}\\\vdots\\\frac{\partial z_c}{\partial W_{ij}}\end{bmatrix}=\underbrace{[0,\cdots, x_j,\cdots, 0]^T}_{\mbox{第$i$个元素为$x_j$}}=x_j \vec{e}_i\end{equation}$$
其中$x_j$是向量$x$的第$j$个元素，为标量，它出现在第$i$行。
综合$(1),(2),(3)$，我们有
$$ \begin{aligned}\frac{\partial E}{\partial W_{ij}}&=tr\bigg(\big( (diag(\hat{y})-\hat{y}\hat{y}^T)\cdot  (diag(\hat{y}))^{-1}  \cdot (-y) \big)^T\cdot x_j \vec{e}_i \bigg)\\&=tr\bigg(\big( \hat{y}\cdot (1_c^Ty)-y\big)^T\cdot x_j \vec{e}_i \bigg)\\&=(\hat{y}-y)^T\cdot x_j \vec{e}_i=r_ix_j\end{aligned}$$
其中$r_i=(\hat{y}-y)_i$表示残差向量的第$i$项

我们可以把上式改写为矩阵形式：
$$ \frac{\partial E}{\partial W}=(\hat{y}-y)\cdot x^T $$
同理可得
$$ \frac{\partial E}{\partial b}=(\hat{y}-y) $$
那么在进行随机梯度下降的时候，更新式就是：
$$ \begin{aligned}&W \leftarrow W - \lambda (\hat{y}-y)\cdot x^T \\&b \leftarrow b - \lambda (\hat{y}-y)\end{aligned}$$
其中$\lambda$是学习率

### 一些个人思考
1. 如何理解每个超平面的得分$\exp(\mathbf{\theta}_k^Tx+b_k)$？
我的理解是$d=\mathbf{\theta}_k^Tx+b_k$表示点$x$到超平面$\mathbf{\theta}_k$，$b_k$的距离（并不完全严谨，类似SVM中的functional margin）。对于logistic回归，如果$d>0$


### 参考
[Matrix Calculus Wiki](https://en.wikipedia.org/wiki/Matrix_calculus)